In [1]:
task = "lenofstay"

In [2]:
import pickle

with open(f'/data/pj20/exp_data/ccscm_ccsproc/sample_dataset_mimic3_{task}_th015.pkl', 'rb') as f:
    sample_dataset = pickle.load(f)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mask_ratio = [
    0.1,
    0.2,
    0.3,
    # 0.4,
    # 0.5,
    # 0.6,
    # 0.7,
    # 0.8,
    # 0.9,
]

In [5]:
from copy import deepcopy
from data_prepare import *


def process(sample_dataset, task):
    dataset_instance = deepcopy(sample_dataset)

    print("Loading embeddings...")
    ent2id, rel2id, ent_emb, rel_emb = load_embeddings(task)
    if task == "drugrec":
        dataset_instance = prepare_drug_indices(dataset_instance)

    map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel = clustering(task, ent_emb, rel_emb, threshold=0.15, load_cluster=True, save_cluster=False)

    print("Processing graph...")
    G = process_graph("mimic3", task, dataset_instance, ent2id, rel2id, map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel, save_graph=False)
    G_tg = from_networkx(G)
    
    print("Processing dataset...")
    dataset_instance = process_sample_dataset("mimic3", task, dataset_instance, G_tg, ent2id, rel2id, map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel, save_dataset=False)

    return G, dataset_instance

In [6]:
import random

for i in range(3):
    for ratio in mask_ratio:
        dataset_instance = deepcopy(sample_dataset)
        remove_ratio = 1 - ratio
        print("ratio: ", ratio)

        conditions_to_remove = random.sample(dataset_instance.get_all_tokens('conditions'), int(len(dataset_instance.get_all_tokens('conditions')) * remove_ratio))
        procedures_to_remove = random.sample(dataset_instance.get_all_tokens('procedures'), int(len(dataset_instance.get_all_tokens('procedures')) * remove_ratio))
        drugs_to_remove = random.sample(dataset_instance.get_all_tokens('drugs'), int(len(dataset_instance.get_all_tokens('drugs')) * remove_ratio))

        for patient in dataset_instance:
            for visit in range(len(patient['conditions'])):
                # relabel the tokens to remove as "0"
                patient['conditions'][visit] = [c if c not in conditions_to_remove else "0" for c in patient['conditions'][visit]]
                patient['procedures'][visit] = [p if p not in procedures_to_remove else "0" for p in patient['procedures'][visit]]
                patient['drugs'][visit] = [d if d not in drugs_to_remove else "0" for d in patient['drugs'][visit]]

        G, dataset_instance = process(dataset_instance, task)

        with open(f'/data/pj20/exp_data/ccscm_ccsproc/sample_dataset_mimic3_{task}_th015_mask_{ratio}_{i}.pkl', 'wb') as f:
            pickle.dump(dataset_instance, f)
        with open(f'/data/pj20/exp_data/ccscm_ccsproc/graph_mimic3_{task}_th015_mask_{ratio}_{i}.pkl', 'wb') as f:
            pickle.dump(G, f)

ratio:  0.1
Loading embeddings...
Processing graph...


100%|██████████| 44399/44399 [00:23<00:00, 1875.03it/s]


Processing dataset...


100%|██████████| 44399/44399 [00:26<00:00, 1658.89it/s]


ratio:  0.2
Loading embeddings...
Processing graph...


...

100%|██████████| 44399/44399 [00:38<00:00, 1165.01it/s]


ratio:  0.3
Loading embeddings...
Processing graph...


100%|██████████| 44399/44399 [00:39<00:00, 1111.82it/s]


Processing dataset...


100%|██████████| 44399/44399 [00:43<00:00, 1026.22it/s]


In [ ]:
import random
from collections import defaultdict

def process(d):
    new_d = defaultdict(list)
    for key, value in d.items():
        if key == "conditions" or key == "procedures" or key == "drugs" or key == "label" or key == "patient_id" or key == "visit_id":
            new_d[key] = value
    return new_d

In [ ]:
import pickle

for j in range(3):
    for ratio in voc_ratios:
        with open(f'/data/pj20/exp_data/ccscm_ccsproc_atc3/sample_dataset_mimic3_{task}_th015_mask_{ratio}_{j}.pkl', 'rb') as f:
            sample_dataset = pickle.load(f)
    
        sample_dataset = list(sample_dataset)
        for i in range(len(sample_dataset)):
            sample_dataset[i] = process(sample_dataset[i])
        
        with open(f'/data/pj20/exp_data/ccscm_ccsproc_atc3/sample_dataset_mimic3_{task}_th015_mask_{ratio}_{j}_pyhealth.pkl', 'wb') as f:
            pickle.dump(sample_dataset, f)